In [ ]:
-- Dla każdego klienta podaj imię i nazwisko pracownika, który w 1997r obsłużył
-- najwięcej jego zamówień, podaj także liczbę tych zamówień (jeśli jest kilku takich
-- pracownikow to wystarczy podać imię nazwisko jednego nich). Za datę obsłużenia
-- zamówienia należy przyjąć orderdate. Zbiór wynikowy powinien zawierać nazwę
-- klienta, imię i nazwisko pracownika oraz liczbę obsłużonych zamówień. (baza
-- northwind)
USE Northwind;
SELECT c.CompanyName, e.FirstName, e.LastName, COUNT(*)
FROM Orders o
JOIN Customers c ON c.CustomerID=o.CustomerID
JOIN Employees e ON e.EmployeeID=o.EmployeeID
GROUP BY c.CompanyName, e.FirstName, e.LastName
HAVING COUNT(*) = (SELECT MAX(temp.count)
                    FROM (SELECT c.CompanyName AS comnam, e.FirstName AS firnam, e.LastName AS lasnam, COUNT(*) AS count
                            FROM Orders o
                            JOIN Customers c ON c.CustomerID=o.CustomerID
                            JOIN Employees e ON e.EmployeeID=o.EmployeeID
                            WHERE YEAR(o.OrderDate)=1997
                            GROUP BY c.CompanyName, e.FirstName, e.LastName) temp
                    WHERE temp.comnam=c.CompanyName)


In [ ]:
-- Podaj liczbę̨ zamówień oraz wartość zamówień (bez opłaty za przesyłkę)
-- obsłużonych przez każdego pracownika w marcu 1997. Za datę obsłużenia
-- zamówienia należy uznać datę jego złożenia (orderdate). Jeśli pracownik nie
-- obsłużył w tym okresie żadnego zamówienia, to też powinien pojawić się na liście
-- (liczba obsłużonych zamówień oraz ich wartość jest w takim przypadku równa 0).
-- Zbiór wynikowy powinien zawierać: imię i nazwisko pracownika, liczbę obsłużonych
-- zamówień, wartość obsłużonych zamówień, oraz datę najpóźniejszego zamówienia
-- (w badanym okresie). (baza northwind)

SELECT
    e.FirstName,
    e.LastName,
    COALESCE(COUNT(o.OrderID), 0) AS NumOrders,
    COALESCE(SUM(od.Quantity * od.UnitPrice * (1 - od.Discount)), 0) AS TotalOrderValue,
    COALESCE(MAX(o.OrderDate), NULL) AS LatestOrderDate
FROM
    Employees e
LEFT OUTER JOIN
    Orders o ON o.EmployeeID = e.EmployeeID
           AND YEAR(o.OrderDate) = 1997
           AND MONTH(o.OrderDate) = 1
LEFT OUTER JOIN
    [Order Details] od ON od.OrderID = o.OrderID
GROUP BY
    e.FirstName, e.LastName;


In [ ]:
-- Podaj liczbę̨ zamówień oraz wartość zamówień (uwzględnij opłatę za przesyłkę)
-- obsłużonych przez każdego pracownika w lutym 1997. Za datę obsłużenia
-- zamówienia należy uznać datę jego złożenia (orderdate). Jeśli pracownik nie
-- obsłużył w tym okresie żadnego zamówienia, to też powinien pojawić się na liście
-- (liczba obsłużonych zamówień oraz ich wartość jest w takim przypadku równa 0).
-- Zbiór wynikowy powinien zawierać: imię i nazwisko pracownika, liczbę obsłużonych
-- zamówień, wartość obsłużonych zamówień. (baza northwind)

SELECT e.FirstName, e.LastName, COUNT(*), CASE WHEN SUM(od.Quantity*od.UnitPrice+o.Freight)>0 THEN SUM(od.Quantity*od.UnitPrice+o.Freight) ELSE 0 END
FROM Employees e 
JOIN Orders o ON e.EmployeeID=o.EmployeeID
JOIN [Order Details] od ON o.OrderID=od.OrderID
WHERE MONTH(o.OrderDate)=2 AND YEAR(o.OrderDate)=1997
GROUP BY e.FirstName, e.LastName

In [ ]:
-- Podaj listę dzieci będących członkami biblioteki, które w dniu '2001-12-14'
-- zwróciły do biblioteki książkę o tytule 'Walking'. Zbiór wynikowy powinien zawierać
-- imię i nazwisko oraz dane adresowe dziecka. (baza library)

USE library;

SELECT j.member_no, l.in_date
FROM juvenile j
JOIN loanhist l ON j.member_no = l.member_no
WHERE l.title_no IN (
    SELECT t.title_no
    FROM title t
    WHERE t.title LIKE '%Walking%'
)  AND l.in_date LIKE '%2001-12-14%';



In [ ]:
-- Podaj listę dzieci będących członkami biblioteki, które w dniu '2001-12-14' nie
-- zwróciły do biblioteki książki o tytule 'Walking'. Zbiór wynikowy powinien zawierać
-- imię i nazwisko oraz dane adresowe dziecka.

USE library;

SELECT m.firstname, m.lastname, a.addres
FROM juvenile j
JOIN member m ON j.member_no = m.member_no 
JOIN (
    SELECT member_no AS amn, CONCAT(street, ', ', city, ', ', state) AS addres
    FROM adult
) a ON a.amn=j.adult_member_no
WHERE
    m.member_no NOT IN (
        SELECT j.member_no
        FROM juvenile j
        JOIN loan l ON l.member_no = j.member_no
        JOIN title t ON t.title_no = l.title_no
        JOIN loanhist lh ON l.copy_no = lh.copy_no
        WHERE YEAR(lh.in_date) = 2001 AND MONTH(lh.in_date) = 12 AND DAY(lh.in_date) = 14
        GROUP BY j.member_no
    );

In [ ]:
-- Podaj tytuły książek zarezerwowanych przez dorosłych członków biblioteki
-- mieszkających w Arizonie (AZ). Zbiór wynikowy powinien zawierać imię i nazwisko
-- członka biblioteki, jego adres oraz tytuł zarezerwowanej książki. Jeśli jakaś osoba
-- dorosła mieszkająca w Arizonie nie ma zarezerwowanej żadnej książki to też
-- powinna znaleźć się na liście, a w polu przeznaczonym na tytuł książki powinien
-- pojawić się napis BRAK. (baza library)

USE library;

SELECT
    m.firstname,
    m.lastname,
    COALESCE(t.title, 'BRAK') AS reserved_title,
    a.street
FROM
    member m
JOIN
    adult a ON m.member_no = a.member_no
LEFT OUTER JOIN
    loan l ON a.member_no = l.member_no
LEFT OUTER JOIN
    title t ON l.title_no = t.title_no
WHERE
    a.state = 'AZ';

In [ ]:
-- Zad.3. Wyświetl podsumowanie zamówień (całkowita cena + fracht) obsłużonych
-- przez pracowników w lutym 1997 roku, uwzględnij wszystkich, nawet jeśli suma
-- wyniosła 0.

SELECT e.FirstName, e.LastName, COALESCE(COUNT(o.OrderDate), 0) ordercount, COALESCE(SUM(od.Quantity*od.UnitPrice*(1-od.Discount)+o.Freight),0) valuesum
FROM Employees e
LEFT OUTER JOIN Orders o ON e.EmployeeID=o.EmployeeID  AND YEAR(o.OrderDate)=1997 AND MONTH(o.OrderDate)=2
LEFT OUTER JOIN [Order Details] od ON o.OrderID=od.OrderID
GROUP BY e.FirstName, e.LastName
ORDER BY ordercount DESC

In [ ]:
-- Wybierz nazwy i numery telefonów klientów, którzy nie kupowali produktów z kategorii Confections.

SELECT CompanyName, Phone
FROM Customers 
WHERE CustomerID NOT IN (SELECT CustomerID
    FROM Orders
    WHERE OrderID IN ( SELECT OrderID
        FROM [Order Details]
        WHERE ProductID IN (SELECT ProductID
            FROM Products
            WHERE CategoryID IN (SELECT CategoryID
                FROM Categories
                WHERE CategoryName LIKE 'Confections'))))


In [ ]:
--Podaj listę dzieci będących członkami biblioteki, które w dniu '2001-12-14'
--zwróciły do biblioteki książkę o tytule 'Walking'. Zbiór wynikowy powinien zawierać
--imię i nazwisko oraz dane adresowe dziecka. (baza library

USE library;

SELECT m.firstname, m.lastname, CONCAT(a.street,',', a.city, ', ', a.state) as adress
FROM juvenile j
JOIN member m ON m.member_no=j.member_no
JOIN adult a ON j.adult_member_no=a.member_no
WHERE j.member_no IN ( SELECT DISTINCT lh.member_no 
                        FROM loanhist lh
                        JOIN title t ON t.title_no=lh.title_no
                        WHERE YEAR(in_date) =2001 AND MONTH(in_date) =12 AND DAY(in_date) =14 AND t.title='Walking')

In [ ]:
-- Zad.2. Wyświetl wszystkich członków biblioteki (imię i nazwisko, adres) 
-- rozróżniając dorosłych i dzieci (dla dorosłych podaj liczbę dzieci),
-- którzy nigdy nie wypożyczyli książki
USE library;

SELECT m.member_no, a.member_no
FROM member m 
CROSS JOIN adult a

In [ ]:
--Zadanko dla Kamilka

USE Northwind;
SELECT o.CustomerID customerid, c.CategoryName categoryname, COUNT(c.CategoryID) 
FROM Orders o 
JOIN [Order Details] od ON o.OrderID = od.OrderId
JOIN Products p ON od.ProductID=p.ProductID
JOIN Categories c ON p.CategoryID=c.CategoryID
GROUP BY o.CustomerID, c.CategoryName
HAVING COUNT(c.CategoryID) = (SELECT MAX(cnt)
                FROM ( SELECT o.CustomerID id, c.CategoryName catname, COUNT(c.CategoryID) cnt
                    FROM Orders o 
                    JOIN [Order Details] od ON o.OrderID = od.OrderId
                    JOIN Products p ON od.ProductID=p.ProductID
                    JOIN Categories c ON p.CategoryID=c.CategoryID
                    GROUP BY o.CustomerID, c.CategoryName) tmp
                 WHERE customerid=tmp.id)
ORDER BY customerid, categoryname, COUNT(c.CategoryID) 



In [ ]:
-- Podaj tytuły książek zarezerwowanych przez dorosłych członków biblioteki
-- mieszkających w Arizonie (AZ). Zbiór wynikowy powinien zawierać imię i nazwisko
-- członka biblioteki, jego adres oraz tytuł zarezerwowanej książki. Jeśli jakaś osoba
-- dorosła mieszkająca w Arizonie nie ma zarezerwowanej żadnej książki to też
-- powinna znaleźć się na liście, a w polu przeznaczonym na tytuł książki powinien
-- pojawić się napis BRAK. (baza library)

USE library;

SELECT m.firstname, m.lastname, COALESCE(t.title,'BRAK') titles
FROM adult a
JOIN member m ON a.member_no=m.member_no
LEFT OUTER JOIN loan l ON l.member_no=a.member_no
LEFT OUTER JOIN title t ON t.title_no=l.title_no
WHERE a.state = 'AZ'


In [ ]:
-- Pokaż nazwy produktów, kategorii 'Beverages' które nie były kupowane w okresie
-- od '1997.02.20' do '1997.02.25' Dla każdego takiego produktu podaj jego nazwę,
-- nazwę dostawcy (supplier), oraz nazwę kategorii. Zbiór wynikowy powinien
-- zawierać nazwę produktu, nazwę dostawcy oraz nazwę kategorii. (baza northwind)
USE Northwind;
SELECT p.ProductName, s.CompanyName, c.CategoryName
FROM Products p
JOIN Suppliers s ON p.SupplierID=s.SupplierID
JOIN Categories c ON c.CategoryID=p.CategoryID
WHERE c.CategoryName LIKE 'Beverages' AND p.ProductID NOT IN (SELECT od.ProductID
                                                            FROM Orders o
                                                            JOIN [Order Details] od ON o.OrderID=od.OrderID
                                                            WHERE o.OrderDate BETWEEN '1997.02.20' AND '1997.02.25')

In [92]:
-- Dla każdego pracownika podaj nazwę klienta, dla którego dany pracownik w 1997r
-- obsłużył najwięcej zamówień, podaj także liczbę tych zamówień (jeśli jest kilku
-- takich klientów to wystarczy podać nazwę jednego nich). Za datę obsłużenia
-- zamówienia należy przyjąć orderdate. Zbiór wynikowy powinien zawierać imię i
-- nazwisko pracownika, nazwę klienta, oraz liczbę obsłużonych zamówień. (baza
-- northwind)

USE Northwind;

SELECT CONCAT(e.FirstName,' ', e.LastName) fullname, c.CompanyName,  COALESCE(COUNT(*),0) number_of_orders
FROM Employees e 
LEFT OUTER JOIN Orders o ON e.EmployeeID=o.EmployeeID AND YEAR(o.OrderDate) =1997
LEFT OUTER JOIN Customers c ON o.CustomerID=c.CustomerID
GROUP BY CONCAT(e.FirstName,' ', e.LastName), c.CompanyName
HAVING COUNT(*) = ( SELECT MAX(no_orders)
                            FROM (SELECT CONCAT(Employees.FirstName,' ', Employees.LastName) empname, Orders.CustomerID custid,  COUNT(*) no_orders
                                    FROM Employees 
                                    LEFT OUTER JOIN Orders ON Employees.EmployeeID=Orders.EmployeeID AND YEAR(Orders.OrderDate) =1997
                                    GROUP BY Employees.EmployeeID, Orders.CustomerID) tmp
                            WHERE CONCAT(e.FirstName,' ', e.LastName)=tmp.empname)



: Msg 8120, Level 16, State 1, Line 16
Column 'Employees.FirstName' is invalid in the select list because it is not contained in either an aggregate function or the GROUP BY clause.

: Msg 8120, Level 16, State 1, Line 16
Column 'Employees.LastName' is invalid in the select list because it is not contained in either an aggregate function or the GROUP BY clause.

Total execution time: 00:00:00.011